# Chains in LangChain


## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

#!pip install pandas


In [5]:
import pandas as pd
df = pd.read_csv('Data.csv')

df.head()

,Unnamed: 0,Product,Review
0,0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain


In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [18]:
llm = ChatOpenAI(temperature=0.9)

In [19]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [20]:
chain = LLMChain(llm=llm, prompt=prompt)

In [22]:
product = "Queen Size Sheet Set"
chain.run(product)

'RoyalRest'

## SimpleSequentialChain


In [24]:
from langchain.chains import SimpleSequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9)


In [26]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [27]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [28]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"Royal Comfort Bedding"
Royal Comfort Bedding: Providing luxurious and comfortable bedding solutions for the ultimate sleep experience fit for royalty.

> Finished chain.


'Royal Comfort Bedding: Providing luxurious and comfortable bedding solutions for the ultimate sleep experience fit for royalty.'

## SequentialChain
When we have multiple inputs and outputs

In [40]:
from langchain.chains import SequentialChain

In [41]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

In [42]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

In [43]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [53]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: English"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [54]:
# prompt template 5: translate follow-up message
fifth_prompt = ChatPromptTemplate.from_template(
    "Translate the following message to {language}:"
    "\n\nMessage : {followup_message}"
)
# chain 5: input= summary, language and output= followup_message
chain_five = LLMChain(llm=llm, prompt=fifth_prompt,
                      output_key="followup_message_translation"
                     )

In [55]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four, chain_five],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "language", "followup_message", 'followup_message_translation'],
    verbose=True
)

In [57]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste poor. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better...\nOld batch or counterfeit!?",
 'summary': "The reviewer finds the taste of the product to be poor and suspects that it may be an old batch or counterfeit because the foam doesn't hold and the taste is noticeably different compared to the ones purchased in stores.",
 'language': 'The review is in French.',
 'followup_message': 'Dear Customer,\n\nThank you for bringing this matter to our attention. We sincerely apologize for any inconvenience you may have experienced with the taste of our product. \n\nWe assure you that we take product quality and customer satisfaction very seriously. Our team has thoroughly investigated your concerns and we can confirm that we strictly adh

In [56]:
review = "به نظر من غذا خوشمزه بود ولی خیلی سرد سرو شده بود. انگار غذا رو از یخچال اورده بودن بیرون! در کل بهتون پیشنهاد نمیکنم"
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'به نظر من غذا خوشمزه بود ولی خیلی سرد سرو شده بود. انگار غذا رو از یخچال اورده بودن بیرون! در کل بهتون پیشنهاد نمیکنم',
 'English_Review': 'In my opinion, the food was delicious but served very cold. It felt like they had taken the food out of the refrigerator! Overall, I do not recommend it to you.',
 'summary': 'The reviewer found the food at the restaurant to be tasty but strongly advises against it due to the disappointing temperature at which it was served.',
 'language': 'The language of the review is Persian (Farsi).',
 'followup_message': 'Dear Reviewer,\n\nThank you for your feedback on our restaurant. We appreciate your positive comments regarding the taste of our food. However, we sincerely apologize for the disappointing temperature at which it was served.\n\nWe understand how important it is for dishes to be served at the right temperature to enhance the dining experience. We have taken note of your feedback and will address this issue immediately with our kitc

## Router Chain


In [58]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [59]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [60]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [61]:
llm = ChatOpenAI(temperature=0)

In [63]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [67]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [70]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [71]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [72]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation refers to the electromagnetic radiation emitted by an object that absorbs all incident radiation and reflects or transmits none. It is called "black body" because it absorbs all wavelengths of light, appearing black at room temperature. \n\nAccording to Planck\'s law, black body radiation is characterized by a continuous spectrum of wavelengths and intensities, which depend on the temperature of the object. As the temperature increases, the peak intensity of the radiation shifts to shorter wavelengths, resulting in a change in color from red to orange, yellow, white, and eventually blue at very high temperatures.\n\nBlack body radiation is a fundamental concept in physics and has significant applications in various fields, including astrophysics, thermodynamics, and quantum mechanics. It played a crucial role in the development of quantum theory, as Max Planck\'s explanation of black body radiation led to the birth of quantum mechanics.'

In [73]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'Thank you for your kind words! As a mathematician, I can definitely help you with your question.\n\nThe problem you presented is a simple addition problem: 2 + 2. To solve it, we can add the two numbers together:\n\n2 + 2 = 4\n\nTherefore, the answer to the question "what is 2 + 2?" is 4.'

In [77]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information necessary for the synthesis of proteins, which are essential for the structure and function of cells. It serves as a blueprint for the production of specific proteins that determine the characteristics and traits of an organism. Additionally, DNA is responsible for the transmission of genetic information from one generation to the next during reproduction. Therefore, every cell in our body contains DNA to ensure the proper functioning and continuity of life.'